In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from loader import *
from models import *
from utils import *
from plotting import *

In [ ]:
im_size_width, im_size_height = 640, 512

plankton_small = stationary_ellipsoid_plankton(im_size_height, im_size_width, radius=(1.5e-7, 9e-7, 1.5e-7), label=0)

plankton_big = stationary_ellipsoid_plankton(im_size_height, im_size_width, radius=(2e-7, 18e-7, 2e-7), label=0)

gradient_amp = 0.5
microscope = plankton_brightfield(im_size_height, im_size_width, gradient_amp)

# no_of_planktons = 50
# sample = create_sample(plankton, no_of_planktons)

no_of_small = lambda: np.random.randint(25, 50)
no_of_big = lambda: np.random.randint(25, 50)

sample = plankton_small**no_of_small + plankton_big**no_of_big


noise_amp = 4
norm_min, norm_max= -0.2, 1.2
image = create_image(noise_amp, sample, microscope, norm_min, norm_max)

plot_image(image)

In [ ]:
label = get_target_image(image.resolve())
label_function = get_target_image
plot_label(label_function, image)

In [ ]:
batch_function = create_custom_batch_function(image, 
                                              outputs=[0], 
                                              function_img=[Normalize_image])

In [ ]:
from deeptrack.generators import ContinuousGenerator
generator = ContinuousGenerator(
    image,
    get_target_image,
    batch_function,
    batch_size=8,
    min_data_size=128,
    max_data_size=512
    )

In [ ]:
no_of_inputs, number_of_outputs = 1, 2
model = generate_unet(im_size_height, im_size_width, no_of_inputs, number_of_outputs)
model = train_model_early_stopping(model, generator, patience=10, epochs=200, steps_per_epoch=10)

In [ ]:
# save_path_model = 'E:\\Documents\\Anaconda\\Jupyterkod\\Exjobb\\Egen kod\\Exjobb\\New imulations\\Copepods\\models\\one_frame.keras'
# model.save(save_path_model)

In [ ]:
load_path_model = 'E:\\Documents\\Anaconda\\Jupyterkod\\Exjobb\\Egen kod\\Exjobb\\New imulations\\Copepods\\models\\one_frame.keras'
model = keras.models.load_model(load_path_model, custom_objects={'softmax_categorical':softmax_categorical})

In [ ]:
folder_path = 'E:\\Documents\\Anaconda\\Jupyterkod\\Exjobb\\Egen kod\\Exjobb\\From erik\\copepods'
im_stack = get_image_stack(
    outputs=[0], 
    folder_path=folder_path, 
    frame_im0=30, 
    im_size_width=640, 
    im_size_height=512,
    im_resize_width=640, 
    im_resize_height=512,
    function_img=[Normalize_image, RemoveRunningMean, Normalize_image],
    function_diff=[Normalize_image],
    path_folder=folder_path,
    tot_no_of_frames=50, 
    im_height=512,
    im_width=640)


plot_image_stack(im_stack)
# plot_prediction(model=model, im_stack=im_stack)
plt.figure(figsize=(10,10))
plt.imshow(model.predict(im_stack)[0,:,:,1]>0.7, cmap='gray')

In [ ]:
positions = extract_positions(
    no_of_frames=100, 
    outputs=[0], 
    folder_path=folder_path, 
    frame_im0=0, 
    im_size_width=640, 
    im_size_height=512,
    im_resize_width=640, 
    im_resize_height=512,
    model=model, 
    layer=1, 
    value_threshold=0.7,
    function_img=[Normalize_image, RemoveRunningMean, Normalize_image],
    function_diff=[Normalize_image],
    path_folder=folder_path,
    tot_no_of_frames=50, 
    im_height=512,
    im_width=640
)


In [ ]:
list_of_plankton = assign_positions_to_planktons(positions, max_dist=30, threshold = 5, extrapolate=True)
list_of_plankton = Interpolate_gaps_in_plankton_positions(list_of_plankton=list_of_plankton)
list_of_plankton = Trim_list_from_stationary_planktons(list_of_plankton=list_of_plankton, min_distance=5)

plankton_track, plankton_dont_track = split_plankton(percentage_threshold=0, list_of_plankton=list_of_plankton)

In [ ]:
plot_and_save_track(no_of_frames=100,
                    plankton_track=plankton_track,
                    plankton_dont_track=plankton_dont_track,
                    folder_path=folder_path,
                    frame_im0=0,
                    save_images=1,
                    show_plankton_track = True,
                    show_plankton_dont_track = 0,
                    show_numbers_track = 0,
                    show_numbers_dont_track = 0,
                    show_numbers_specific_plankton = False,
                    show_specific_plankton = False,
                    specific_plankton = None,
                    color_plankton_track = 'b',
                    color_plankton_dont_track = 'r',
                    color_specific_plankton = 'w',
                    save_path = 'E:\\Documents\\Anaconda\\Jupyterkod\\Exjobb\\Egen kod\\Exjobb\\New imulations\\Copepods\\track',
                    frame_name = 'track',
                    file_type = '.jpg')

In [ ]:
Make_video(frame_im0=0,
           folder_path='E:\\Documents\\Anaconda\\Jupyterkod\\Exjobb\\Egen kod\\Exjobb\\New imulations\\Copepods\\track', 
           save_path='E:\\Documents\\Anaconda\\Jupyterkod\\Exjobb\\Egen kod\\Exjobb\\New imulations\\Copepods\\Test_video.avi', 
           fps=7, 
           no_of_frames=100)